In [1]:
# python.exe -m pip install --upgrade pip
import numpy as np
import opendatasets as od
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from unidecode import unidecode

In [2]:
cadena_con_tildes = "Canción con acentos y tildes"
cadena_sin_tildes = unidecode(cadena_con_tildes)

print(cadena_sin_tildes)

Cancion con acentos y tildes


In [3]:
# od.download("https://www.kaggle.com/datasets/shivd24coder/fifa-world-cup-dataset")

In [4]:
attendence_df = pd.read_csv(r'fifa-world-cup-dataset\attendence.csv')
goals_df = pd.read_csv(r'fifa-world-cup-dataset\Goals.csv')
match_df = pd.read_csv(r'fifa-world-cup-dataset\match.csv')
team_df = pd.read_csv(r'fifa-world-cup-dataset\Team.csv')

print('Attendence\n')
print(attendence_df.info(),2*'\n', 50*'*','\n')
print('Goals\n')
print(goals_df.info(), 2*'\n', 50*'*','\n')
print('Match\n')
print(match_df.info(), 2*'\n', 50*'*','\n')
print('Team\n')
print(team_df.info())

Attendence

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 836 entries, 0 to 835
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ATTENDANCE  836 non-null    object
 1   DATE        836 non-null    object
 2   PK          22 non-null     object
 3   REFEREE     836 non-null    object
 4   STADIUM     836 non-null    object
 5   game_id     836 non-null    int64 
 6   stage       836 non-null    object
 7   team1       822 non-null    object
 8   team2       822 non-null    object
 9   url         836 non-null    object
dtypes: int64(1), object(9)
memory usage: 65.4+ KB
None 

 ************************************************** 

Goals

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2207 entries, 0 to 2206
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   game_id     2207 non-null   int64 
 1   minute      2207 non-null   object
 2   player      2

In [5]:
attendence_df.head()

,ATTENDANCE,DATE,PK,REFEREE,STADIUM,game_id,stage,team1,team2,url
0,25000,27-05-1934 (16:00 h),NaN,RenÃ© Mercet (SWI),Stadio Nazionale del PNF (Roma),1,FIRST ROUND,Italy,USA,1934_ITALY_FS.htm
1,16000,27-05-1934 (16:30 h),NaN,John van Moorsel (NED),Benito Mussolini (Torino),2,FIRST ROUND,Austria,France,1934_ITALY_FS.htm
2,8000,27-05-1934 (16:30 h),NaN,Francesco Mattea (ITA),Giovanni Berta (Firenze),3,FIRST ROUND,Germany,Belgium,1934_ITALY_FS.htm
3,9000,27-05-1934 (16:30 h),NaN,Jean Langenus (BEL),Stadio del Littorio (Trieste),4,FIRST ROUND,Czechoslovakia,Romania,1934_ITALY_FS.htm
4,33000,27-05-1934 (16:30 h),NaN,Ivan Eklind (SWE),San Siro (Milano),5,FIRST ROUND,Switzerland,Netherlands,1934_ITALY_FS.htm


In [6]:
goals_df.head()

,game_id,minute,player,team1score,team2score
0,1,18,Schiavio,1,0
1,1,20,Orsi,2,0
2,1,29,Schiavio,3,0
3,1,57,Donelli,3,1
4,1,63,Ferrari,4,1


In [7]:
match_df.head()

,edition,round,score,team1,team2,url,venue,year
0,1930-URUGUAY,GROUP_STAGE,4-1 (3-0),France,Mexico (México),1930_URUGUAY_FS.htm#1-WC-30-I,Montevideo.,1930
1,1930-URUGUAY,GROUP_STAGE,3-0 (2-0),USA,Belgium (België),1930_URUGUAY_FS.htm#13-WC-30-I,Montevideo.,1930
2,1930-URUGUAY,GROUP_STAGE,2-1 (2-0),Yugoslavia (Југославија),Brazil (Brasil),1930_URUGUAY_FS.htm#7-WC-30-I,Montevideo.,1930
3,1930-URUGUAY,GROUP_STAGE,3-1 (1-0),Romania (România),Peru (Perú),1930_URUGUAY_FS.htm#10-WC-30-I,Montevideo.,1930
4,1930-URUGUAY,GROUP_STAGE,1-0 (0-0),Argentina,France,1930_URUGUAY_FS.htm#2-WC-30-I,Montevideo.,1930


In [8]:
team_df.head()

,No,Pos,Player,DOB/Age,Caps,Club,Country,ClubCountry,Year
0,1,1GK,Ángel Bossio,(1905-05-05)5 May 1905 (aged 25),NaN,Talleres,Argentina,Argentina,1930
1,1,1GK,Juan Botasso,(1908-10-23)23 October 1908 (aged 21),NaN,Quilmes,Argentina,Argentina,1930
2,9,4FW,Roberto Cherro,(1907-02-23)23 February 1907 (aged 23),NaN,Boca Juniors,Argentina,Argentina,1930
3,4,2DF,Alberto Chividini,(1907-02-23)23 February 1907 (aged 23),NaN,Central Norte Tucumán,Argentina,Argentina,1930
4,10,4FW,Attilio Demaría,(1909-03-19)19 March 1909 (aged 21),NaN,Estudiantil Porteño,Argentina,Argentina,1930


In [9]:
def convert_to_integer(value):
    if value.isdigit():
        return int(value)
    elif str(type(value)) in ['int', 'float']:
        return value
    return 0

def date(cadena):
    cadena = str(cadena)
    if cadena == 'nan':
        return np.nan
    patron_1 = '(\d{1,2})-(\d{1,2})-(\d{4}) \((\d{1,2}):(\d{1,2}) h\)'
    match_1 = re.search(patron_1, cadena)
    
    patron_2 = '(\d{1,2})-(\d{1,2})-(\d{4})'
    match_2 = re.search(patron_2, cadena)

    if match_1 is not None:
        fecha = pd.to_datetime(cadena.strip(), format='%d-%m-%Y (%H:%M h)')
    elif match_2 is not None:
        indice_inicial = cadena.find('(')
        indice_final = cadena.find(')')+1
        fecha = cadena.replace(cadena[indice_inicial:indice_final], '')
        fecha = pd.to_datetime(fecha.strip(), format='%d-%m-%Y')
    else:
        return np.nan
    return fecha

def string_and_strip(value):
    if value == 'nan':
        return np.nan
    else:
        return str(value).strip()

def encode_decode(cadena):
    if str(cadena) == 'nan':
        return np.nan
    else:
        cadena_corregida = str(cadena).encode('latin-1').decode('utf-8')
        cadena_corregida = unidecode(cadena_corregida)
        return cadena_corregida

def stage_string_correction(cadena):
    cadena_corregida = cadena.encode('iso-8859-1').decode('utf-8')
    cadena_corregida = cadena_corregida.replace('\r\n', '').strip()
    palabras = cadena_corregida.split()  # Divide la cadena en palabras
    cadena_corregida = ' '.join(palabras)  # Une las palabras con un solo espacio como separador
    return cadena_corregida

def extraer_texto_entre_parentesis(cadena):
    if type(cadena) == str:
        indice_inicial = cadena.find('(')
        indice_final = cadena.find(')')
        if indice_inicial >= 0:
            texto_entre_parentesis = cadena[indice_inicial + 1 :indice_final]
            return texto_entre_parentesis.strip()
        else:
            return np.nan
    else:
        return cadena
    
    
def extraer_texto_fuera_de_parentesis(cadena):
    if type(cadena) == str:
        indice_inicial = cadena.find('(')
        indice_final = cadena.find(')')
        if indice_inicial >= 0:
            texto_fuera_de_parentesis = cadena.replace(cadena[indice_inicial:indice_final + 1], '')
            return texto_fuera_de_parentesis.strip()
        else:
            return cadena.strip()
    else:
        return cadena


In [10]:
attendence_df_work = attendence_df.copy()

In [11]:
attendence_df_work['ATTENDANCE'] = attendence_df_work['ATTENDANCE'].apply(convert_to_integer)
attendence_df_work['DATE'] = attendence_df_work['DATE'].apply(date)
attendence_df_work['PK'] = attendence_df_work['PK'].apply(string_and_strip).apply(encode_decode)
attendence_df_work['REFEREE'] = attendence_df_work['REFEREE'].apply(encode_decode)
attendence_df_work['STADIUM'] = attendence_df_work['STADIUM'].apply(encode_decode)
attendence_df_work['stage'] = attendence_df_work['stage'].apply(stage_string_correction)
attendence_df_work['team1'] = attendence_df_work['team1'].apply(encode_decode)
attendence_df_work['team2'] = attendence_df_work['team2'].apply(encode_decode)

In [12]:
attendence_df_work.nlargest(20, columns='ATTENDANCE')

,ATTENDANCE,DATE,PK,REFEREE,STADIUM,game_id,stage,team1,team2,url
309,179854,1950-07-16 15:00:00,NaN,George Reader (ENG),Maracana (Rio de Janeiro),310,FINAL ROUND,Uruguay,Brazil,1950_BRAZIL_FS.htm
307,152772,1950-07-13 15:00:00,NaN,Reginald Leafe (ENG),Maracana (Rio de Janeiro),308,FINAL ROUND,Brazil,Spain,1950_BRAZIL_FS.htm
298,142429,1950-07-01 15:00:00,NaN,Mervyn Griffiths (WAL),Maracana (Rio de Janeiro),299,GROUP STAGE,Brazil,Yugoslavia,1950_BRAZIL_FS.htm
304,138886,1950-07-09 15:00:00,NaN,Arthur Ellis (ENG),Maracana (Rio de Janeiro),305,FINAL ROUND,Brazil,Sweden,1950_BRAZIL_FS.htm
236,114600,1986-06-07 12:00:00,NaN,George Courtney (ENG),Estadio Azteca (Mexico D.F.),237,GROUP STAGE,Mexico,Paraguay,1986_MEXICO_FS.htm
254,114580,1986-06-15 12:00:00,NaN,Romualdo Arppi Filho (BRA),Estadio Azteca (Mexico D.F.),255,1/8 FINAL,Mexico,Bulgaria,1986_MEXICO_FS.htm
264,114580,1986-06-22 12:00:00,NaN,Ali Ben Nasser (TUN),Estadio Azteca (Mexico D.F.),265,1/4 FINAL,Argentina,England,1986_MEXICO_FS.htm
269,114580,1986-06-29 12:00:00,NaN,Romualdo Arppi Filho (BRA),Estadio Azteca (Mexico D.F.),270,FINAL,Argentina,FRG,1986_MEXICO_FS.htm
267,110420,1986-06-25 16:00:00,NaN,Antonio Marquez Ramirez (MEX),Estadio Azteca (Mexico D.F.),268,1/2 FINAL,Argentina,Belgium,1986_MEXICO_FS.htm
224,110000,1986-06-03 12:00:00,NaN,Carlos Alfonso Esposito (ARG),Estadio Azteca (Mexico D.F.),225,GROUP STAGE,Belgium,Mexico,1986_MEXICO_FS.htm


In [13]:
attendence_df_work.loc[attendence_df['PK'].notnull()]

,ATTENDANCE,DATE,PK,REFEREE,STADIUM,game_id,stage,team1,team2,url
101,71000,1982-07-08 21:00:00,0-1 (Giresse); 1-1 (Kaltz) / 1-2 (Amoros); 2-2...,Charles Corver (NED),Ramon Sanchez Pizjuan (Sevilla),102,1/2 FINAL,FRG,France,1982_SPAIN_FS.htm
262,65677,1986-06-21 12:00:00,0-0 (Socrates [saved]); 0-1 (Stopyra) / 1-1 (A...,Ion Igna (ROM),Estadio Jalisco (Guadalajara),263,1/4 FINAL,Brazil,France,1986_MEXICO_FS.htm
263,44386,1986-06-21 16:00:00,1-0 (Allofs); 1-1 (Negrete) / 2-1 (Brehme); 2-...,Jesus Diaz Palacios (COL),Estadio Universitario (Monterrey),264,1/4 FINAL,FRG,Mexico,1986_MEXICO_FS.htm
265,45000,1986-06-22 16:00:00,1-0 (Senor); 1-1 (Claesen) / 1-1 (Eloy [saved]...,Siegfried Kirschen (GDR),Estadio Cuauhtemoc (Puebla),266,1/4 FINAL,Spain,Belgium,1986_MEXICO_FS.htm
353,71030,1994-07-05 16:30:00,0-0 (Garcia Aspe [out]); 0-0 (Balakov [saved])...,Jamal al-Sharif (SYR),"Giants Stadium (New Jersey, New York)",354,1/8 FINAL,Mexico,Bulgaria,1994_USA_FS.htm
357,81715,1994-07-10 12:30:00,0-0 (Mild [out]); 1-0 (Raducioiu) / 1-1 (K. An...,Philip Don (ENG),"Stanford Stadium (Palo Alto, San Francisco)",358,1/4 FINAL,Romania,Sweden,1994_USA_FS.htm
361,94194,1994-07-17 12:30:00,0-0 (Baresi [out]); 0-0 (Marcio Santos [saved]...,Sandor Puhl (HUN),"Rose Bowl (Pasadena, Los Angeles)",362,FINAL,Brazil,Italy,1994_USA_FS.htm
417,35500,1998-06-30 21:00:00,1-0 (Berti); 1-1 (Shearer) / 1-1 (Crespo [save...,Kim Milton Nielsen (DEN),Geoffroy Guichard (Saint-Etienne),418,1/8 FINAL,Argentina,England,1998_FRANCE_FS.htm
418,77000,1998-07-03 16:30:00,0-1 (Zidane); 1-1 (R. Baggio) / 1-1 (Lizarazu ...,Hugh Dallas (SCO),"Stade de France (Saint-Denis , Paris)",419,1/4 FINAL,Italy,France,1998_FRANCE_FS.htm
422,54000,1998-07-07 21:00:00,1-0 (Ronaldo); 1-1 (F. de Boer) / 2-1 (Rivaldo...,Ali Mohammed Bujsaim (UAE),Velodrome (Marseille),423,1/2 FINAL,Brazil,Netherlands,1998_FRANCE_FS.htm


In [14]:
goals_df_work = goals_df.copy()
goals_df_work['player'] = goals_df_work['player'].apply(encode_decode)

In [15]:
match_df_work = match_df.copy()

match_df_work['team1_name'] = match_df_work['team1'].apply(extraer_texto_entre_parentesis)
match_df_work['team2_name'] = match_df_work['team2'].apply(extraer_texto_entre_parentesis)

match_df_work['team1'] = match_df_work['team1'].apply(extraer_texto_fuera_de_parentesis)
match_df_work['team2'] = match_df_work['team2'].apply(extraer_texto_fuera_de_parentesis)

In [16]:
match_df_work

,edition,round,score,team1,team2,url,venue,year,team1_name,team2_name
0,1930-URUGUAY,GROUP_STAGE,4-1 (3-0),France,Mexico,1930_URUGUAY_FS.htm#1-WC-30-I,Montevideo.,1930,NaN,México
1,1930-URUGUAY,GROUP_STAGE,3-0 (2-0),USA,Belgium,1930_URUGUAY_FS.htm#13-WC-30-I,Montevideo.,1930,NaN,België
2,1930-URUGUAY,GROUP_STAGE,2-1 (2-0),Yugoslavia,Brazil,1930_URUGUAY_FS.htm#7-WC-30-I,Montevideo.,1930,Југославија,Brasil
3,1930-URUGUAY,GROUP_STAGE,3-1 (1-0),Romania,Peru,1930_URUGUAY_FS.htm#10-WC-30-I,Montevideo.,1930,România,Perú
4,1930-URUGUAY,GROUP_STAGE,1-0 (0-0),Argentina,France,1930_URUGUAY_FS.htm#2-WC-30-I,Montevideo.,1930,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
7294,2014-BRAZIL,1/4_FINAL,(C),3,4,2014_BRAZIL_FS.htm#832-WC-14-III,Salvador.,2014,NaN,NaN
7295,2014-BRAZIL,1/2_FINAL,(X),A,B,2014_BRAZIL_FS.htm#833-WC-14-IV,Belo Horizonte.,2014,NaN,NaN
7296,2014-BRAZIL,1/2_FINAL,(Y),C,D,2014_BRAZIL_FS.htm#834-WC-14-IV,São_Paulo.,2014,NaN,NaN
7297,2014-BRAZIL,PLACES_3&4,xxx,LOSER X,LOSER Y,2014_BRAZIL_FS.htm#835-WC-14-V,Brasília.,2014,NaN,NaN


In [17]:
team_df_work = team_df.copy()

team_df_work['DOB_num'] = team_df_work['DOB/Age'].apply(extraer_texto_entre_parentesis)
team_df_work['DOB/Age'] = team_df_work['DOB/Age'].apply(extraer_texto_fuera_de_parentesis)
team_df_work['Age'] = team_df_work['DOB/Age'].apply(extraer_texto_entre_parentesis)
team_df_work['DOB/Age'] = team_df_work['DOB/Age'].apply(extraer_texto_fuera_de_parentesis)

In [18]:
team_df_work 

,No,Pos,Player,DOB/Age,Caps,Club,Country,ClubCountry,Year,DOB_num,Age
0,1,1GK,Ángel Bossio,5 May 1905,NaN,Talleres,Argentina,Argentina,1930,1905-05-05,aged 25
1,1,1GK,Juan Botasso,23 October 1908,NaN,Quilmes,Argentina,Argentina,1930,1908-10-23,aged 21
2,9,4FW,Roberto Cherro,23 February 1907,NaN,Boca Juniors,Argentina,Argentina,1930,1907-02-23,aged 23
3,4,2DF,Alberto Chividini,23 February 1907,NaN,Central Norte Tucumán,Argentina,Argentina,1930,1907-02-23,aged 23
4,10,4FW,Attilio Demaría,19 March 1909,NaN,Estudiantil Porteño,Argentina,Argentina,1930,1909-03-19,aged 21
...,...,...,...,...,...,...,...,...,...,...,...
8892,-,4FW,Park Chu-Young,10 July 1985,62,Watford,South Korea,England,2014,1985-07-10,aged 28
8893,-,4FW,Ji Dong-Won,28 May 1991,26,FC Augsburg,South Korea,Germany,2014,1991-05-28,aged 23
8894,-,4FW,Lee Keun-Ho,11 April 1985,62,Sangju Sangmu,South Korea,South Korea,2014,1985-04-11,aged 29
8895,-,4FW,Koo Ja-Cheol,27 February 1989,35,Mainz 05,South Korea,Germany,2014,1989-02-27,aged 25
